In [1]:
# https://modin.readthedocs.io/en/stable/flow/modin/config.html

In [2]:
# https://modin.readthedocs.io/en/stable/flow/modin/config.html

In [1]:
# Dependency versions can be modified here if necessary

# !pip install -I git+https://github.com/cloudera/cmlextensions.git --quiet
# !pip install -I ray[client]==2.6.0 --quiet

In [1]:
import cmlextensions_custom.src.cmlextensions.ray_cluster as rc
import os
import cmlapi
import json
from pprint import pprint
import ray

In [2]:
HOST = os.getenv("CDSW_API_URL").split(":")[0] + "://" + os.getenv("CDSW_DOMAIN")
USERNAME = os.getenv("CDSW_PROJECT_URL").split("/")[6]  # args.username  # "vdibia"
API_KEY = os.getenv("CDSW_APIV2_KEY")
PROJECT_NAME = os.getenv("CDSW_PROJECT")
PROJECT_ID=os.getenv("CDSW_PROJECT_ID")

cml = cmlapi.default_client(url=HOST,cml_api_key=API_KEY)

def set_environ(Cml,Item,Value):
    Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
    if Project.environment=='':
        Project_Environment={}
    else:
        Project_Environment=json.loads(Project.environment)
    Project_Environment[Item]=Value
    Project.environment=json.dumps(Project_Environment)
    Cml.update_project(Project,project_id=os.getenv("CDSW_PROJECT_ID"))

def get_environ(Cml,Item):
    Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
    Project_Environment=json.loads(Project.environment)
    return Project_Environment[Item]

In [3]:
#head_system_configs='{"object_spilling_config":"{\"type\":\"filesystem\",\"params\":{\"directory_path\":\"/tmp\"}}"}'

In [4]:
cluster = rc.RayCluster( num_workers=30,
                         worker_cpu=32, worker_memory=128, worker_nvidia_gpu=0, 
                         head_cpu=32, head_memory=128, head_nvidia_gpu=0
                       )
cluster.init(7200)
#set_environ(cml,"RAY_ADDRESS",cluster.get_client_url())

Starting ray head...
Starting 30 ray workers...

--------------------
Ray cluster started
--------------------

The Ray dashboard is running at
https://y7yn91fp4yobswep.ml-f534c58f-a06.se-sandb.a465-9q4k.cloudera.site/

To connect to this Ray cluster from this CML Session,
use the following Python code:
  import ray
  ray.init(address='ray://100.100.126.213:10001')



In [5]:
os.environ["RAY_memory_monitor_refresh_ms"] = "0"
os.environ["AUTOSCALER_MAX_NUM_FAILURES"] = "inf"
os.environ['GRPC_VERBOSITY']='DEBUG'
os.environ['http_proxy']=''
os.environ['https_proxy']=''
os.environ["MODIN_PROGRESS_BAR"] = "True"  # Ray Modin Bar

In [6]:
# # Set MODIN_MEMORY to half of your total RAM (in bytes) - adjust based on your setup
#os.environ["MODIN_MEMORY"] = str(22 * 1024 * 1024 * 1024) 

# # Set RAY_OBJECT_STORE_SIZE to a lower value if needed
#os.environ["RAY_OBJECT_STORE_SIZE"] = str(8 * 1024 * 1024 * 1024)  # Example for 8 GB

In [7]:
import modin.config

#print(modin.config.Memory.get())
#modin.config.Memory.put(str(1200 * 1024 * 1024 * 1024)) 
#print(modin.config.Memory.get())

The following initializes a Ray cluster, which can be customized to the size of workload you need to run. To troubleshoot starting up the Ray cluster, refer to the README

In [8]:
#ray.init(address=cluster.get_client_url())

In [9]:
runtime_env = {
    'env_vars': {
        "RAY_memory_monitor_refresh_ms": "0",
        "AUTOSCALER_MAX_NUM_FAILURES":"inf"
    }
}

ray.init(
    runtime_env=runtime_env,
    address=cluster.get_client_url(),
    _system_config={
       "object_spilling_config": json.dumps(
            {"type": "filesystem", "params": {"directory_path": "/tmp/spill"}}, 
        ), #"automatic_object_spilling_enabled": True,
    },
)

W0000 00:00:1723782615.696579     393 log.cc:142] Log level DEBUG is not suitable for production. Prefer WARNING or ERROR. However if you see this message in a debug environmenmt or test environmenmt it is safe to ignore this message.


ClientContext(dashboard_url='127.0.0.1:8090', python_version='3.10.13', ray_version='2.6.0', ray_commit='0db82e31e249eac614f7c8e7da1c4f8f05c9064a', protocol_version='2023-06-27', _num_clients=1, _context_to_restore=<ray.util.client._ClientContext object at 0x7f8f26cbb7c0>)

In [10]:
#import modin.config
#modin.config.NPartitions.put(16)
#print(modin.config.NPartitions.get()) # prints '16'

In [11]:
#shm_size = os.statvfs('/dev/shm').f_frsize * os.statvfs('/dev/shm').f_bavail
#print(f"Available /dev/shm size: {shm_size / 1024 / 1024:.2f} MB")

In [ ]:
import modin.pandas as pd
import numpy as np
x = 4
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**19))) for _ in range(x)], axis=1) # x*2GB frames -- Working!
df.info()

In [ ]:
df.to_parquet("/home/cdsw/onehundredtwentygbdf")

In [ ]:
ray.cluster_resources()

In [13]:
import modin.config as cfg

# Default value for this config is 'False'
print(cfg.RangePartitioning.get()) # False

False


In [14]:
import modin.config as cfg

# Default value for this config is 'False'
print(cfg.RangePartitioning.get()) # False

# Set the config to 'True' inside of the context-manager
#with cfg.context(RangePartitioning=True):
#    print(cfg.RangePartitioning.get()) # True
#    df.merge(...) # will use range-partitioning impl

# Once the context is over, the config gets back to its previous value
print(cfg.RangePartitioning.get()) # False

# You can also set multiple config at once when you pass a dictionary to 'cfg.context'
print(cfg.AsyncReadMode.get()) # False

with cfg.context(RangePartitioning=True, AsyncReadMode=True):
    print(cfg.RangePartitioning.get()) # True
    print(cfg.AsyncReadMode.get()) # True
print(cfg.RangePartitioning.get()) # False
print(cfg.AsyncReadMode.get()) # False

False
False
False
True
True
False
False


In [15]:
import modin.config
import modin.pandas as pd

# Checking initially set `Engine` config,
# which corresponds to `MODIN_ENGINE` environment
# variable
print(modin.config.Engine.get()) # prints 'Dask'

# Checking default value of `NPartitions`
print(modin.config.NPartitions.get()) # prints '8'

# Changing value of `NPartitions`
modin.config.NPartitions.put(16)
print(modin.config.NPartitions.get()) # prints '16'

Ray
640
16


In [9]:
# https://modin.readthedocs.io/en/stable/flow/modin/config.html

In [16]:
print(modin.config.ModinNumpy.get())

False


In [7]:
# This may take a few minutes to download
import urllib.request
dataset_url = "https://modin-datasets.intel.com/testing/yellow_tripdata_2015-01.csv"
urllib.request.urlretrieve(dataset_url, "taxi.csv")  

('taxi.csv', <http.client.HTTPMessage at 0x7fb01c0934c0>)

In [8]:
import modin.pandas as pd
import time

In [9]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

start = time.time()

modin_df = pd.read_csv("taxi.csv", parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"], quoting=3)

end = time.time()
modin_duration = end - start
print("Time to read with Modin: {} seconds".format(round(modin_duration, 3)))

Time to read with Modin: 13.626 seconds


In [10]:
n_rows = 100000
row_mult = 80 # widening factor

In [11]:
# modin creation
new_df = modin_df.head(n_rows)
for i in range(row_mult):
    for col in modin_df.columns:
        new_col_name = f"{col}_copy_{i}"
        new_df[new_col_name] = new_df[col]

In [12]:
memory_in_bytes = modin_df.memory_usage(deep=True).sum()
memory_in_megabytes = memory_in_bytes / (1024 ** 2)
print(f"Memory usage: {memory_in_megabytes:.2f} MB")

Memory usage: 401.98 MB


In [13]:
memory_in_bytes = new_df.memory_usage(deep=True).sum()
memory_in_megabytes = memory_in_bytes / (1024 ** 2)
print(f"Memory usage: {memory_in_megabytes:.2f} MB")

ValueError: could not broadcast input array from shape (105,) into shape (2,)

In [ ]:
start_time = time.time()

new_df_pandas = pandas_df.head(n_rows).copy()
for i in range(row_mult):
    for col in pandas_df.columns:
        new_col_name = f"{col}_copy_{i}"
        new_df_pandas[new_col_name] = new_df_pandas[col]
pandas_duration = time.time() - start_time
print(f"Pandas df widening {pandas_duration:.4f} seconds")

In [ ]:
memory_in_bytes = modin_df.memory_usage(deep=True).sum()
memory_in_megabytes = memory_in_bytes / (1024 ** 2)
print(f"Memory usage: {memory_in_megabytes:.2f} MB")

In [7]:
import modin.pandas as pd
import numpy as np
x = 4
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(x)]) # x*2GB frames -- Working!
df.info()

(raylet) Spilled 3072 MiB, 55 objects, write throughput 1542 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4096 MiB, 73 objects, write throughput 1242 MiB/s.
(raylet) [2024-08-14 00:10:25,121 E 289 289] (raylet) local_object_manager.cc:360: Failed to send object spilling request: GrpcUnavailable: RPC Error message: Socket closed; RPC Error details: 
(raylet) [2024-08-14 00:10:26,264 E 289 289] (raylet) local_object_manager.cc:360: Failed to send object spilling request: GrpcUnavailable: RPC Error message: Socket closed; RPC Error details: 
(raylet) [2024-08-14 00:10:27,499 E 289 289] (raylet) local_object_manager.cc:360: Failed to send object spilling request: GrpcUnavailable: RPC Error message: Socket closed; RPC Error details: 
(raylet) [2024-08-14 00:10:28,089 E 289 289] (raylet) local_object_manager.cc:360: Failed to send object spilling request: GrpcUnavailable: RPC Error message: Socket closed; RPC Error details: 
(raylet) [2024-08-14 00:10:28,684 

ConnectionError: Request can't be sent because the Ray client has already been disconnected due to an error. Last exception: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Attempted to reconnect a session that has already been cleaned up"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Attempted to reconnect a session that has already been cleaned up", grpc_status:5, created_time:"2024-08-14T00:11:04.208595648+00:00"}"
>

In [ ]:
#ray.shutdown()
#cluster.terminate()